#### install the requirements

# create venv for this chapter
!python3 -m venv chapter_env

In [ ]:
#actiavate it 
!source chapter_env/bin/activate

In [ ]:
!python3 -m pip install -r requirements.txt

#### importing all lib for this chapter